In [ ]:
!pip install sentence-transformers faiss-cpu transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from google.colab import files

uploaded = files.upload()  # This will prompt you to upload the JSON file

Saving processed_schemes.json to processed_schemes.json


In [ ]:
import json

# Load your JSON file
with open('processed_schemes.json', 'r') as f:
    schemes = json.load(f)

# Preprocess schemes into text chunks
scheme_texts = []
for scheme in schemes:
    text = f"""
    Scheme Name: {scheme['scheme_name']}
    Ministries/Departments: {scheme.get('ministry', 'N/A')}
    Eligibility Criteria: {scheme['eligibility_criteria']}
    Description & Benefits: {scheme['benefits']}
    Application Process: {scheme['application_process']}
    Tags: {', '.join(scheme['tags'])}
    """
    scheme_texts.append(text.strip())

# Now scheme_texts contains the necessary data for RAG processing.

In [ ]:
import json

# Load big JSON
with open('processed_schemes.json', 'r') as f:
    raw_schemes = json.load(f)

# Create minimal JSON
minimal_schemes = []
for scheme in raw_schemes:
    minimal_schemes.append({
        'scheme_name': scheme.get('scheme_name', ''),
        'ministry': scheme.get('ministry', 'N/A'),
        'eligibility_criteria': scheme.get('eligibility_criteria', ''),
        'benefits': scheme.get('benefits', ''),
        'application_process': scheme.get('application_process', ''),
        'tags': scheme.get('tags', []),
    })

# Save compact JSON
with open('schemes_minimal.json', 'w') as f:
    json.dump(minimal_schemes, f, indent=2)

In [ ]:
import json
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import numpy as np

# Load minimal schemes
with open('schemes_minimal.json', 'r') as f:
    schemes = json.load(f)

# Preprocess for embedding
scheme_texts = [f"{scheme['scheme_name']} - {scheme['eligibility_criteria']} {scheme['benefits']} {scheme['application_process']}" for scheme in schemes]

# Embedding Model
embedder = SentenceTransformer('BAAI/bge-small-en')
scheme_embeddings = embedder.encode(scheme_texts, normalize_embeddings=True)

# FAISS index
dimension = scheme_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(scheme_embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
import time
import json
from transformers import pipeline

# --------------------- Configuration ---------------------
MODEL_NAME = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
MAX_HISTORY = 3
SCHEME_DB = {}
SCHEME_TEXTS = []
chat_history = []

# --------------------- Data Loading ---------------------
with open('processed_schemes.json', 'r') as f:
    schemes = json.load(f)

for scheme in schemes:
    key = scheme['scheme_name'].lower().replace(' ', '')
    SCHEME_DB[key] = {
        'name': scheme['scheme_name'],
        'ministry': scheme.get('ministry', 'N/A'),
        'eligibility': [e.strip() for e in scheme['eligibility_criteria'].split('\n') if e.strip()],
        'benefits': [b.strip() for b in scheme['benefits'].split('\n') if b.strip()],
        'application': [a.strip() for a in scheme['application_process'].split('\n') if a.strip()],
        'tags': [t.lower() for t in scheme.get('tags', [])]
    }

    text_chunk = f"""Scheme: {scheme['scheme_name']}
Ministry: {scheme.get('ministry', 'N/A')}
Eligibility: {scheme['eligibility_criteria']}
Benefits: {scheme['benefits']}
Application: {scheme['application_process']}
Tags: {', '.join(scheme.get('tags', []))}"""
    SCHEME_TEXTS.append(text_chunk.strip())

# --------------------- Model Setup ---------------------
generator = pipeline(
    'text-generation',
    model=MODEL_NAME,
    max_new_tokens=1200,
    temperature=0.35,
    repetition_penalty=1.35,
    do_sample=True,
    top_p=0.9
)

# --------------------- Enhanced UI Components ---------------------
header = HTML("""
<div style="
    background: linear-gradient(135deg, #1a237e, #0d47a1);
    padding: 25px;
    border-radius: 8px;
    color: white;
    text-align: center;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    margin-bottom: 20px;
">
<h1 style="margin:0; font-family: 'Helvetica Neue'; font-weight: 300">
     Government Scheme Expert
</h1>
<p style="opacity:0.9; margin:5px 0 0">AI-Powered Scheme Assistance with Memory</p>
</div>
""")

question_input = widgets.Text(
    placeholder='Ask about any government scheme...',
    layout=widgets.Layout(width='100%', padding='12px'),
    style={'description_width': '0', 'font_size': '14px'}
)

submit_button = widgets.Button(
    description="Get Answer",
    button_style='primary',
    icon='fa-commenting',
    layout=widgets.Layout(width='120px', margin='10px 0 0 0')
)

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Processing:',
    style={'bar_color': '#1a237e'},
    layout=widgets.Layout(width='100%', visibility='hidden'),
    bar_style='info'
)

output_area = widgets.Output()

# --------------------- Optimized Core Logic ---------------------
def format_response(scheme, question):
    """Create focused response based on question keywords"""
    question_lower = question.lower()

    # Determine which sections to show
    show_eligibility = any(word in question_lower for word in ['eligibility', 'criteria', 'qualification'])
    show_benefits = any(word in question_lower for word in ['benefit', 'advantage', 'facility'])
    show_application = any(word in question_lower for word in ['apply', 'process', 'procedure', 'step'])

    # If no specific section asked, show all
    if not (show_eligibility or show_benefits or show_application):
        show_eligibility = show_benefits = show_application = True

    response = f"✅ *{scheme['name']}*"

    if show_eligibility:
        eligibilities = '\n'.join(f'• {item}' for item in scheme['eligibility'])
        response += f"\n\n📋 *Eligibility:*\n{eligibilities}"

    if show_benefits:
        benefits = '\n'.join(f'• {item}' for item in scheme['benefits'])
        response += f"\n\n🎁 *Benefits:*\n{benefits}"

    if show_application:
        applications = '\n'.join(f'→ {step}' for step in scheme['application'])
        response += f"\n\n📝 *Application:*\n{applications}"

    return response


def find_best_match(question):
    """Enhanced matching with context awareness"""
    q_lower = question.lower()
    q_clean = question.lower().replace('?', '').strip()

    # 1. Check for context words (this, that, it)
    context_words = {'this', 'that', 'it', 'scheme'}
    if any(word in q_lower for word in context_words) and chat_history:
        # Get last discussed scheme from history
        last_question, last_answer = chat_history[-1]
        for scheme_name in SCHEME_DB:
            if scheme_name in last_question.lower():
                return SCHEME_DB[scheme_name], 95  # High confidence

    # 2. Direct name match
    scheme_name = next((name for name in SCHEME_DB if name in q_lower), None)
    if scheme_name:
        return SCHEME_DB[scheme_name], 90

    # 3. Partial name match
    for name in SCHEME_DB:
        if name in q_clean:
            return SCHEME_DB[name], 85

    # 4. Tag-based search
    matched_tags = []
    for scheme in SCHEME_DB.values():
        common_tags = set(q_clean.split()) & set(scheme['tags'])
        if common_tags:
            matched_tags.append((scheme, len(common_tags)))

    if matched_tags:
        return max(matched_tags, key=lambda x: x[1])[0], 75

    return None, 0

def build_context():
    """Build context from chat history"""
    return "\n\n".join(
        f"Previous Question: {q}\nPrevious Answer: {a[:500]}"
        for q, a in chat_history[-MAX_HISTORY:]
    )

def generate_answer(question):
    """Generate focused answer with context awareness"""
    # Get last discussed scheme from history
    last_scheme = None
    if chat_history:
        for q, a in reversed(chat_history):
            for scheme_name in SCHEME_DB:
                if scheme_name in q.lower():
                    last_scheme = SCHEME_DB[scheme_name]
                    break
            if last_scheme:
                break

    # Find best match with context
    scheme, confidence = find_best_match(question)

    # Use last scheme if context words detected
    if not scheme and last_scheme:
        context_trigger_words = {'this', 'that', 'it', 'scheme'}
        if any(word in question.lower() for word in context_trigger_words):
            scheme = last_scheme
            confidence = 90

    if confidence > 70:
        return format_response(scheme, question)

    # Fallback to LLM with strict instructions
    prompt = f"""<system>
Focus strictly on the asked section.
Previous Discussion: {chat_history[-1][0] if chat_history else "None"}
Answer ONLY about: {chat_history[-1][0] if chat_history else "General schemes"}
Follow these rules:
1. If asked about benefits → only list benefits
2. If asked about eligibility → only eligibility criteria
3. Use bullet points (• for main points, → for subpoints)
4. Maximum 5 points
5. Never add extra sections
6. Use simple English

Current Context: {build_context()}
</system>

Question: {question}
Answer:"""

    response = generator(
        prompt,
        max_new_tokens=400,
        temperature=0.2,
        do_sample=True
    )[0]['generated_text'].split('Answer:')[-1].strip()

    # Update history
    chat_history.append((question, response))
    if len(chat_history) > MAX_HISTORY:
        chat_history.pop(0)

    return response
# --------------------- Improved Interaction Flow ---------------------
def on_submit(_):
    with output_area:
        question = question_input.value.strip()

        if not question:
            display(HTML("<div style='color:red'>Please enter a question!</div>"))
            return

        # UI State Management
        question_input.disabled = True
        submit_button.disabled = True
        progress_bar.layout.visibility = 'visible'

        # Progress animation
        for i in range(10):
            progress_bar.value += 10
            time.sleep(0.02)

        try:
            answer = generate_answer(question)
            display(Markdown(f"""
<div style="
    padding: 15px;
    margin: 10px 0;
    background: #f8f9fa;
    border-radius: 8px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
">
    <div style="color: #1a237e; font-weight:600; margin-bottom:8px">❓ {question}</div>
    <div style="color: #2c3e50; line-height:1.5">{answer}</div>
</div>
"""))
        except Exception as e:
            display(HTML(f"<div style='color:red'>Error: {str(e)}</div>"))

        # Reset UI
        question_input.value = ""
        question_input.disabled = False
        submit_button.disabled = False
        progress_bar.value = 0
        progress_bar.layout.visibility = 'hidden'

# --------------------- Final Setup ---------------------
submit_button.on_click(on_submit)
display(header)
display(question_input)
display(submit_button)
display(progress_bar)
display(output_area)

Device set to use cpu


Text(value='', layout=Layout(padding='12px', width='100%'), placeholder='Ask about any government scheme...', …

Button(button_style='primary', description='Get Answer', icon='commenting', layout=Layout(margin='10px 0 0 0',…

IntProgress(value=0, bar_style='info', description='Processing:', layout=Layout(visibility='hidden', width='10…

Output()

In [ ]:
!git clone https://github.com/SuyashGupta24/MyScheme-QnA-System.git

Cloning into 'MyScheme-QnA-System'...


In [ ]:
%cd MyScheme-QnA-System

/content/MyScheme-QnA-System


In [ ]:
!git checkout -b main


Switched to a new branch 'main'


In [ ]:
!git config --global user.email "23cs3064@rgipt.ac.in"
!git config --global user.name "SuyashGupta24"


In [ ]:
!git checkout -b my-new-branch


Switched to a new branch 'my-new-branch'


In [ ]:
!git branch -m my-new-branch

In [ ]:
!git add .
!git commit -m "Initial commit"


On branch my-new-branch

Initial commit

nothing to commit (create/copy files and use "git add" to track)


In [ ]:
!git push origin my-new-branch



error: src refspec my-new-branch does not match any
error: failed to push some refs to 'https://github.com/SuyashGupta24/MyScheme-QnA-System.git'
